In [ ]:
def calculate_ibae_parameters(M):
    """
    Calculates the total parameter count for the IBAE model as a function of M.
    Based on Table 1 and experimental settings (n=4, l=9, N=20).
    """
    # Hyperparameters from experimental setup
    n = 4   # Oversampling rate
    l = 9   # Output length multiplier
    N = 20  # Channel memory parameter
    
    # 1. Transmitter (Three layers: M -> 4M -> 4M -> n)
    # Weights + Biases
    tx_params = (M * 4*M + 4*M) + (4*M * 4*M + 4*M) + (4*M * n + n)
    
    # 2. IBCM (Two tributaries, each with two hidden layers)
    # Input nodes: (2N + 1) * n = (41 * 4) = 164
    ibcm_in = (2*N + 1) * n
    # Tributary 1 (Linear): 164 -> 33n -> 9n
    trib1 = (ibcm_in * 33*n + 33*n) + (33*n * 9*n + 9*n)
    # Tributary 2 (ReLU): 164 -> 33n -> 9n
    trib2 = (ibcm_in * 33*n + 33*n) + (33*n * 9*n + 9*n)
    # Final concatenation to Output (l*n = 36)
    # The paper describes Tributaries added together, so the "final" weights 
    # are often represented as a single linear combination to the output.
    ibcm_out_layer = (9*n * l*n + l*n)
    ibcm_params = trib1 + trib2 + ibcm_out_layer
    
    # 3. Receiver (Three layers: l*n -> 4M -> 4M -> M)
    rx_in = l*n # 36
    rx_params = (rx_in * 4*M + 4*M) + (4*M * 4*M + 4*M) + (4*M * M + M)
    
    return {
        "Transmitter": tx_params,
        "IBCM": ibcm_params,
        "Receiver": rx_params,
        "Total": tx_params + ibcm_params + rx_params
    }

# Example for M = 64
counts = calculate_ibae_parameters(64)
print(f"Total Parameters for M=64: {counts['Total']:,}")
print(f"Breakdown: {counts}")